In [8]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split


In [9]:
housing = pd.read_csv('datasets/CaliforniaHousing/cal_housing.data', names=['longitude', 'latitude', 'housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue'])

In [10]:
housing.head()

,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [11]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
longitude           20640 non-null float64
latitude            20640 non-null float64
housingMedianAge    20640 non-null float64
totalRooms          20640 non-null float64
totalBedrooms       20640 non-null float64
population          20640 non-null float64
households          20640 non-null float64
medianIncome        20640 non-null float64
medianHouseValue    20640 non-null float64
dtypes: float64(9)
memory usage: 1.4 MB


In [12]:
target = housing['medianHouseValue']
data = housing.drop('medianHouseValue', axis=1)

data['avgBedrooms'] = data['totalBedrooms'] / data['households']
data['avgRooms'] = data['totalRooms'] / data['households']
data['person_per_household'] = data['population'] / data['households']

data = data.drop(['totalRooms', 'totalBedrooms'], axis=1)
data.head()

,longitude,latitude,housingMedianAge,population,households,medianIncome,avgBedrooms,avgRooms,person_per_household
0,-122.23,37.88,41.0,322.0,126.0,8.3252,1.023810,6.984127,2.555556
1,-122.22,37.86,21.0,2401.0,1138.0,8.3014,0.971880,6.238137,2.109842
2,-122.24,37.85,52.0,496.0,177.0,7.2574,1.073446,8.288136,2.802260
3,-122.25,37.85,52.0,558.0,219.0,5.6431,1.073059,5.817352,2.547945
4,-122.25,37.85,52.0,565.0,259.0,3.8462,1.081081,6.281853,2.181467


In [16]:
X = data
y = target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2019)

ss = StandardScaler()

X_train = ss.fit_transform(X_train)

X_test = ss.transform(X_test)

sgdr = SGDRegressor(verbose=False, max_iter=100,random_state=2019)

sgdr.fit(X_train, y_train)

y_pred = sgdr.predict(X_test)

print r2_score(y_test, y_pred)


0.632017716822608


In [19]:
print np.shape(X_train), np.shape(X_test)

(15480, 9) (5160, 9)


In [ ]:
n_epochs = 100
learning_rate = 1e-2

X_train = tf.constant(data, dtype=tf.float32, name='X')
y = tf.constant(np.array(target), dtype=tf.float32, name='y')

In [ ]:
X


In [ ]:
y

In [ ]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')

In [ ]:
y_pred = tf.matmul(X, theta, name='predictions')

In [ ]:
error = y_pred - y

In [ ]:
mae = tf.reduce_mean(tf.abs(error), name='mae')
mse = tf.reduce_mean(tf.square(error), name='mse')

In [ ]:
gradients = tf.gradients(mse, [theta])[0]

In [ ]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print mae.eval()
            #print theta.eval()
        sess.run(training_op)
    best_theta = theta.eval()

In [ ]:
y_pred

In [ ]:
y